# Physics Informed Neural Networks <br> F1 Car Front Wing Aerodymanics

## PINN

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import csv
from pinn import PINN

In [ ]:
in_filepath = "/Users/ggito/repos/pinns/data/"
points_filename = "front_wing_points_final.csv"
norms_filename = "front_wing_norms_final.csv"

wing_df = pd.read_csv(in_filepath + points_filename)
norm_df = pd.read_csv(in_filepath + norms_filename)

print(wing_df)
print(norm_df)

In [ ]:
x_max = 1
y_max = 1
z_max = 1
t_max = 1

Nx = 10
Ny = 10
Nz = 10
Nt = 10

dx = x_max / (Nx - 1)
dy = y_max / (Ny - 1)
dz = z_max / (Nz - 1)
dt = t_max / (Nt - 1)

x_test = np.linspace(0, x_max, Nx)
y_test = np.linspace(0, y_max, Ny)
z_test = np.linspace(0, z_max, Nz)
t_test = np.linspace(0, t_max, Nt)

x_grid, y_grid, z_gripd, t_grid = np.meshgrid(x_test, y_test, z_test, t_test)

In [ ]:
input_dim = 4
output_dim = 4
# hidden_units = [32, 32, 32]
# hidden_units = [64, 64, 64, 64]
# hidden_units = [128, 128, 128, 128]
# hidden_units = [256, 256, 256, 256]
# hidden_units = [512, 512]
hidden_units = [1024, 1024, 1024]
# hidden_units = [20, 40, 80, 100, 100, 80, 40, 20]

if torch.backends.mps.is_available():
  device = torch.device("mps")
  x = torch.ones(1, device=device)
  print(x)
else:
  print("MPS device not found.")
  device = "cpu"

# device = "cpu"

pinn = PINN(input_dim, output_dim, hidden_units).to(device)

In [ ]:
# optimizer = torch.optim.Adam(pinn.parameters(), lr=0.001)
optimizer = torch.optim.LBFGS(pinn.parameters())

epochs = 1000
Nf = Nx   # num of collocation points -> pde evaluation -> Nf^4... needs fixing: sample Nf points from the whole 3D domain
N0 = Ny   # num of points to evaluate initial conditons -> N0^4
Nb = Nx   # num of points to evaluate boundary conditions -> Nb^4
Nw = Nx   # num of points of the surface of the front wing to evaluate boundary conditions -> Nw^4

# Density (rho): 1.184 kg/m³
# Dynamic viscosity (mu): 1.81e-5 kg/m.s
rho = 1184
mu = 1.81e-5

log_filepath = "/Users/ggito/repos/pinns/data/log.csv"

In [ ]:
def sample_points_in_domain(min, max, num_of_samples):
  return torch.tensor(np.random.uniform(min, max, size=(num_of_samples, 1)), dtype=torch.float32, device=device, requires_grad=True)

In [ ]:
def zeros(num):
  return torch.zeros(size=(num, 1), dtype=torch.float32, device=device, requires_grad=True)

In [ ]:
def closure(report_losses=False):
  optimizer.zero_grad()

  # TODO: use quasi monte carlo sampling
  z_f = sample_points_in_domain(0, x_max, Nf)
  y_f = sample_points_in_domain(0, y_max, Nf)
  x_f = sample_points_in_domain(0, z_max, Nf)
  t_f = sample_points_in_domain(0, t_max, Nf)

  x0 = sample_points_in_domain(dx, x_max, N0)
  y0 = sample_points_in_domain(dy, y_max, N0)
  z0 = sample_points_in_domain(dz, z_max, N0)
  t0 = sample_points_in_domain(0, t_max, N0)

  x_b = zeros(Nb)
  y_b = sample_points_in_domain(0, y_max, Nb)
  z_b = sample_points_in_domain(0, z_max, Nb)
  t_b = sample_points_in_domain(0, t_max, Nb)
  
  # sample Nw wing points
  sampled_indices = wing_df.sample(n=Nw).index

  x_w, y_w, z_w = [torch.tensor(wing_df.loc[sampled_indices, col].values, dtype=torch.float32, device=device, requires_grad=True) for col in ['x', 'y', 'z']]
  t_w = sample_points_in_domain(0, t_max, Nw)

  total_loss, pde_loss, ic_loss, bc_loss, wing_loss = pinn.loss(
                  x_f, y_f, z_f, t_f,
                  x0, y0, z0, t0,
                  x_b, y_b, z_b, t_b,
                  x_w, y_w, z_w, t_w,
                  mu, rho, dt,
                  0.005, 0.25, 0.25, 0.25,
                  log_filepath)

  total_loss.backward()

  if report_losses:
      return total_loss, pde_loss, ic_loss, bc_loss, wing_loss
  else:
      return total_loss

for epoch in range(epochs):
  optimizer.step(closure)
  # if epoch % 10 == 0:
  total_loss, pde_loss, ic_loss, bc_loss, wing_loss = closure(report_losses=True)
  print(f'Epoch: {epoch},\tTotal loss: {total_loss.item()},\tPDE loss: {pde_loss.item()},\tIC loss: {ic_loss.item()},\tBC loss: {bc_loss.item()},\tWing loss: {wing_loss.item()}')

In [ ]:
out_filepath = "/Users/ggito/repos/pinns/data/"
torch.save(pinn, out_filepath + 'pinn.pt')

In [ ]:
loaded_model = torch.load(out_filepath + 'pinn.pt')
loaded_model.eval()